### Merger Rate of Black Holes in a Primordial Black Hole Cluster

###### Code Writed by Tadeo D.
###### Dedicated to the Moon of my life.
**References:** [arXiv:2105.14523](https://arxiv.org/pdf/2105.14523.pdf).

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import math
import seaborn as sns
from scipy import stats
import csv
from scipy import constants
from scipy.integrate import odeint
from tqdm import tqdm
from scipy import integrate
from scipy.integrate import quad, dblquad
from astropy.cosmology import Planck18 as cosmo# WMAP9 as cosmo #
from astropy import constants as const
import astropy.units as u
#from astropy.cosmology import WMAP9, z_at_value
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import cm
#pl.rcParams['figure.figsize'] = (16, 9)
#pl.style.use('ggplot')
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#
#from mlxtend.plotting import plot_linear_regression
from sympy import *
from sympy import simplify

In [2]:
H0 = cosmo.H(0)  #cosmo.H(0)  <Quantity 67.66 km / (Mpc s)>
G = const.G
c = const.c
z_r = 5000 #3450
z_eq = 3411
z_0 = 0
z_init = 6.900e9
rho_c0 = (3*(H0**2))/(8*math.pi*G)
rho_eq = cosmo.critical_density(3411)
rho_crit = cosmo.critical_density(0)
Omega_DM = cosmo.Odm(0)
Omega_M = cosmo.Om(0)
Omega_R = 8.4e-5
Omega_lmb = 1-Omega_M - Omega_R

$$\rho_{i}(r) = \rho_{0,i}\left(\frac{r}{r_{0}}\right)^{-1} \left(1 + \frac{r^{2}}{r^{2}_{0}}\right)^{-2}$$

In [3]:
N_i = 10**4 #M_i/m_i is the number of stars of mass m_i
M = 10**5 * u.M_sun#M is the total mass of the cluster
R = 1.*u.parsec #Scale factor that determines the core radius
v_rel = (G*M/R)**(1./2.)#The characteristic velocity of the PBHs
m_i = 1*u.M_sun
m_j = m_i
Energy = G*(m_i*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy

print("N_i =\t", N_i)
print("M =\t", M)
print("R =\t", R)
print("v_rel =\t", v_rel.to(u.parsec* u.s**(-1)))
print("m_i =\t", m_i)
print("m_j =\t", m_j)
print("E =\t", Energy.to(u.parsec**2.*u.s**(-2.)))

N_i =	 10000
M =	 100000.0 solMass
R =	 1.0 pc
v_rel =	 6.7209397035645e-13 pc / s
m_i =	 1.0 solMass
m_j =	 1.0 solMass
E =	 2.258551524947482e-22 pc2 / s2


In [4]:
sigma = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.))#v_rel**(18./7.))
sigma = sigma.to(u.m**(32./7)/u.s**(18./7.))#.to(u.parsec**2)
#sigma.to(u.AU**2)

In [5]:
def f_i(E): ## eq. 19 shapiro
    return (24. * (2.)**(1./2.) * N_i * R**2. * E**(7./2.))/(7*(math.pi**(3.))*(G**5.)*(M**5.))

def rho_i(r):## eq. 20
    return ((3*M)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

def phi(r):## eq. 21
    return (G*M/R)*((1 + r**(2.)/R**(2.))**(-1./2.))

def integrant(E):
    return E**(7./2.)*((2*(a - E))**(3./2.))

In [6]:
f_i(Energy)
#PH_i(1.*u.parsec)

<Quantity 2.5349244e+20 kg(3/2) s3 / (m(9/2) pc(3/2) solMass(3/2))>

In [7]:
n = (3.*N_i)/(4*math.pi*(R**(3.)))
print(n)

2387.32414637843 1 / pc3


In [8]:
r = np.linspace(.1, 1., 10)
print("r(pc)\t\t phi(r) \t\t\t\t\t\t v_i²(r)")
phi_array=[]
for i in r:
    i=i*u.parsec
    rho_value = rho_i(i)
    phi_value = phi(i).to(u.parsec**(2.)/u.s**2)
    #print(rho_i(i), "\t", phi_value)
    phi_array.append(phi_value.value)
    a = phi_value.value
    I, err = quad(integrant, 0, a)
    constant = ((4.*math.pi*m_i)/rho_value)
    constant2 = (24. * (2.**(1./2.)) * N_i * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    vel = (I*const*(u.parsec**(12.)/(u.s**(12.)))).to(u.km**(2.)/(u.s**(2.)))
    result = ((7*math.pi*(phi_value**6.))/(2.**(17./2.)))
    vel2 = (result*const).to(u.km**(2.)/(u.s**(2.)))
    print("%.2f" %i.value,"\t",phi_value,"\t",(vel**(1./2.)),"\t", (vel2**(1./2.)))
print("\n")
#print(phi_array)

r(pc)		 phi(r) 						 v_i²(r)
0.10 	 4.494685526656458e-25 pc2 / s2 	 4.625782138541091 km / s 	 4.625782439600092 km / s
0.20 	 4.429383960831521e-25 pc2 / s2 	 4.592056082945187 km / s 	 4.5920563818092015 km / s
0.30 	 4.326600034242027e-25 pc2 / s2 	 4.538463988829716 km / s 	 4.538464284205806 km / s
0.40 	 4.194024892154189e-25 pc2 / s2 	 4.468389483283585 km / s 	 4.468389774099028 km / s
0.50 	 4.0402197923747064e-25 pc2 / s2 	 4.3856908531227345 km / s 	 4.385691138555916 km / s
0.60 	 3.8733839099994897e-25 pc2 / s2 	 4.294185401842412 km / s 	 4.29418568132016 km / s
0.70 	 3.7005550067478666e-25 pc2 / s2 	 4.197289754408988 km / s 	 4.197290027580491 km / s
0.80 	 3.527264880702962e-25 pc2 / s2 	 4.097835837982706 km / s 	 4.097836104681468 km / s
0.90 	 3.357536254982149e-25 pc2 / s2 	 3.9980283979679823 km / s 	 3.998028658170993 km / s
1.00 	 3.1940741978991656e-25 pc2 / s2 	 3.8994918714520987 km / s 	 3.8994921252420736 km / s




In [11]:
def n_i(r):
    return (rho_i(r)/m_i)

def n_j(r2):
    return (rho_i(r2)/m_j)

def vel_j(r):
    constant = ((4.*math.pi*m_j)/rho_i(r))
    constant2 = (24. * (2.**(1./2.)) * N_i * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    result = ((7*math.pi*(phi(r)**6.))/(2.**(17./2.)))
    return ((result*const).to(u.km**(2.)/(u.s**(2.))))**(1./2.)

In [18]:
def GAMMA(r1):
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/((R.value)**(2.)))
    return r1**(2.)*((1.+(a))**(-9./4.))*(1./((1. + b)**(1./4.)) + 1./((1. + a)**(1./4.)))**(3./7.) - abs((1./((1. + b)**(1./4.)) - 1./((1. + a)**(1./4.))))**(3./7.)

In [31]:
#CONSTANTS
#sigma = 2*math.pi*(85.*math.pi/(6.*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.))
alfa = ((4*24*(2**(1/2))*N_i*(R**(2.)))/(math.pi*(G**(5.))*(M**(5.))*(2**(17./2.))))
betha = ((3.*M)/(m_i*4*math.pi*(R**(3.))))
etha = (G*M/R)
zetha = alfa**(-1./2.) * betha**(3./2.) * etha**(-3.) * alfa**(3./14.) * etha**(9./7.) * betha**(-3./14.)
#print(zetha)

In [19]:
r = np.linspace(.1, 1., 10)
for i in r:
    r0 = i
    I, err = quad(GAMMA, i-0.1, i+0.1)
    print("%.2f"%r0,"\t",I)

0.10 	 -0.010762505505623694
0.20 	 -0.008672081755758903
0.30 	 -0.0021532899709860117
0.40 	 0.006036504926768599
0.50 	 0.01387967497409903
0.60 	 0.02012870918596568
0.70 	 0.024270834711567477
0.80 	 0.02633219142923157
0.90 	 0.02664021087714669
1.00 	 0.02563121938631204


In [36]:
def GAMMA(r1,r0): #Equation (11)
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)

r = np.linspace(.1, 1., 10)
for i in r:
    r0 = i*u.parsec
    I, err = quad(GAMMA, i-0.1, i+0.1, args=(r0,))
    nash_rubi = (14*math.pi/3.)*sigma*(n_j(r0)/vel_j(r0))*zetha
    #I = I*units
    merger = (I*nash_rubi).to((u.parsec**(-3.)*(u.s**(-1))))
    print("%.2f"%r0.value,"\t",merger)

0.10 	 -3.214533439379688e-18 1 / (pc3 s)
0.20 	 -2.425079622661047e-18 1 / (pc3 s)
0.30 	 -5.417769819662549e-19 1 / (pc3 s)
0.40 	 1.320330743884343e-18 1 / (pc3 s)
0.50 	 2.566016349646108e-18 1 / (pc3 s)
0.60 	 3.078088014125738e-18 1 / (pc3 s)
0.70 	 3.0223395965269048e-18 1 / (pc3 s)
0.80 	 2.642506095838464e-18 1 / (pc3 s)
0.90 	 2.141354236965777e-18 1 / (pc3 s)
1.00 	 1.6458089887969776e-18 1 / (pc3 s)
